In [1]:
#!pip install -q gradio langchain sentence-transformers faiss-cpu transformers

import gradio as gr
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from transformers import pipeline

corpus = [
    "I love this product! It has changed my life.",
    "The service was terrible. I am very disappointed.",
    "Amazing experience, will definitely come back again!",
    "Not worth the money. The quality was poor.",
    "I'm neutral about the update. It didn’t do much."
]

def setup_vector_store(corpus):
    docs = [Document(page_content=t) for t in corpus]
    splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    split_docs = splitter.split_documents(docs)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    return vectorstore.as_retriever(search_kwargs={"k": 2})

retriever = setup_vector_store(corpus)
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def sentiment_rag_interface(query):
    results = retriever.get_relevant_documents(query)
    response = ""
    for i, doc in enumerate(results):
        sentiment = classifier(doc.page_content)[0]
        response += f"Match {i+1}:\n{doc.page_content}\n→ Sentiment: {sentiment['label']} (Confidence: {sentiment['score']:.2f})\n\n"
    return response.strip()

with gr.Blocks(title="💬 Sentiment RAG Analyzer") as demo:
    gr.Markdown("## Ask questions to analyze sentiment from corpus using RAG + Transformers")
    with gr.Row():
        query_input = gr.Textbox(label="Your Question", placeholder="e.g. What do users feel about quality?")
    output_box = gr.Textbox(label="RAG Sentiment Output", lines=8)
    btn = gr.Button("Analyze")

    btn.click(sentiment_rag_interface, inputs=query_input, outputs=output_box)

demo.launch(debug=True)

C:\Users\nares\AppData\Local\Temp\ipykernel_4352\48806955.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
Device set to use cpu


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\nares\AppData\Local\Temp\ipykernel_4352\48806955.py:30: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


Keyboard interruption in main thread... closing server.
